# Code Demonstration

The purpose of this notebook is to demonstrate how to run the alignment codes (MW, Cluster-Match, Cluster-Cluster) introduced in the paper and to illustrate the output that can be extracted from the methods.

All methods are implemented in Python. As demonstrated in this notebook, they can be used as components in a script. Alternatively, they can also be executed as a standalone program.

In the following cells, we import some packages that will be used. 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
basedir = '..'
sys.path.append(basedir)

In [3]:
from IPython.display import display, HTML

Here we import the Aligner class, which will be used to perform alignment via direct matching. We also import the FileLoader class to load the input data, and model parameters in the last line.

In [4]:
from shared_bin_matching import SharedBinMatching as Aligner
from preprocessing import FileLoader
from models import AlignmentHyperPars

## 1. Loading Data

We demonstrate with an example how to perform an aligment of peaks via direct matching (MW). No ionization product clustering information is used yet.

First we need to load the input files. They should be in the CSV format. In each file, each row corresponds to a peak. The columns are the peakID, mass, rt and intensity values of that peak. Every column is separated by comma.

An example file content is given below:

    peakID, mass, rt, intensity
    0, 115.06328573368826, 577.2680053710938, 2.07935616E9
    1, 117.07894145664908, 529.8209838867188, 1.69812992E9
    2, 103.09961983676502, 932.0130004882812, 1.231442304E9
    3, 257.10263614620476, 605.9390258789062, 1.099428992E9
    4, 135.05452246252298, 496.76300048828125, 5.99815168E8
    5, 137.0477186540849, 545.9990234375, 4.568576E8

In the example below, we load three beer files to be aligned. Their path is specified by the *input_dir* parameter. All .txt and .csv files in *input_dir* will be loaded.

In [5]:
loader = FileLoader()
input_dir = '../input/beer3pos'
data_list = loader.load_model_input(input_dir, verbose=True)

Loading files from ../input/beer3pos
7553 features read from /Users/joewandy/git/precursor-alignment/input/beer3pos/beer3-file1.csv
7579 features read from /Users/joewandy/git/precursor-alignment/input/beer3pos/beer3-file2.csv
7240 features read from /Users/joewandy/git/precursor-alignment/input/beer3pos/beer3-file3.csv


To perform alignment, we create an aligner object. This requires the *AlignmentHyperPars* object, which stores the parameters of the model. The following parameters are used by all alignment methods (MW, Cluster-Match, Cluster-Cluster).

- hp.across_file_mass_tol is the mass tolerance across files (for matching)
- hp.across_file_rt_tol is the RT tolerance across file (for matching)

In [6]:
hp = AlignmentHyperPars()
hp.within_file_mass_tol = 3               
hp.within_file_rt_tol = 10

These parameters are required to produce the ionization product clustering of peaks. The resulting IP clusters are used by Cluster-Match and Cluster-Cluster to perform alignment.

- hp.within_file_mass_tol is the mass tolerance within file
- hp.within_file_rt_tol is the RT tolerance within file
- hp.alpha_mass is the uniform Dirichlet hyper-parameter
- hp.mass_clustering_n_iterations is the number of iterations when performing Gibbs sampling

In [7]:
hp.across_file_mass_tol = 6
hp.across_file_rt_tol = 15
hp.alpha_mass = 1
hp.mass_clustering_n_iterations = 1000

The following are additional parameters for alignment using Cluster-Cluster.

- hp.dp_alpha is the concentration parameter for Dirichlet Process for Cluster-Cluster
- hp.beta is the prior on the multinomial when adduct signature is used in Cluster-Cluster
- hp.rt_clustering_nsamps is the number of iterations when performing Gibbs sampling for Cluster-Cluster 
- hp.rt_clustering_burnin is the number of burn-in samples to discard

In [8]:
hp.dp_alpha = 1000.0
hp.beta = 0.1
hp.rt_clustering_nsamps = 1000
hp.rt_clustering_burnin = 500

After loading the files and defining parameters, we can now create the alignment object.

In [9]:
transformation_file = None
aligner_mw = Aligner(data_list, transformation_file, hp)

Hyperparameters across_file_mass_tol=6, across_file_rt_tol=15, alpha_mass=1, beta=0.1, dp_alpha=1000.0, mass_clustering_n_iterations=1000, matching_alpha=0.3, rt_clustering_burnin=500, rt_clustering_nsamps=1000, second_stage_clustering_use_adduct_likelihood=True, second_stage_clustering_use_mass_likelihood=True, second_stage_clustering_use_rt_likelihood=True, within_file_mass_tol=3, within_file_rt_tol=10


## 2. A direct-matching alignment via MW

Here we perform a direct-matching alignment using the aligner object. To do this, we specify *match_mode* to be 0. 

In [10]:
match_mode = 0
aligner_mw.run(match_mode)

Match mode 0
Matching peak features
Processing file 0
Processing file 1
Computing score matrix
Running matching

Processing file 2
Computing score matrix
Running matching



The result of alignment is stored in the *aligner.alignment_results* attribute of the aligner object. Here we see that 12116 aligned peaksets are produced.

In [11]:
print len(aligner_mw.alignment_results)

12547


The cell below demonstrates how we can print the alignment result in a tabular format and also export it to the *demo_mw.csv* file. The columns in the table are:

- *peakset* refers to an aligned peakset (each row in the table sharing the same peakset peaks that have been matched). 
- *file* refers to the file position in the *data_list* (file 0 is the first file)
- *m/z*, *rt* and *intensity* refer to the m/z, RT and intensity of the peak

In [12]:
df = aligner_mw.print_peaksets()
display(df)
df.to_csv('demo_mw.csv', index=False)

,peakset,file,m/z,rt,intensity
0,0,0,159.894077,1327.880005,4.090063e+06
1,0,1,159.894081,1324.469971,4.041590e+06
2,0,2,159.894080,1331.050049,4.540444e+06
3,1,0,133.110248,764.265015,6.634680e+04
4,1,1,133.110261,762.302002,4.652207e+04
5,1,2,133.110215,766.489014,9.011648e+04
6,2,0,95.920470,1424.310059,3.960048e+05
7,2,1,95.920470,1425.510010,3.898136e+05
8,2,2,95.920472,1430.209961,4.934352e+05
9,3,0,285.142249,548.476990,1.645755e+06


## 3. Matching of ionization product clusters via Cluster-Match

In this section, we demonstrate alignment using Cluster-Match. This is a two-stage alignment process. First peaks within each run are clustered using PrecursorCluster, producing **ionisation product** (IP) clusters. In the Cluster-Match method, the IP clusters are matched across runs, and the correspondence of their member peaks are established to produce the final matching result.

For PrecursorCluster, we require a transformation file to be specified. This is encoded in the [YAML format](https://en.wikipedia.org/wiki/YAML).

In [13]:
transformation_file = '../pos_transformations.yml'

In the cell below, another aligner object is created for the Cluster-Match method.

In [14]:
aligner_cm = Aligner(data_list, transformation_file, hp)

Hyperparameters across_file_mass_tol=6, across_file_rt_tol=15, alpha_mass=1, beta=0.1, dp_alpha=1000.0, mass_clustering_n_iterations=1000, matching_alpha=0.3, rt_clustering_burnin=500, rt_clustering_nsamps=1000, second_stage_clustering_use_adduct_likelihood=True, second_stage_clustering_use_mass_likelihood=True, second_stage_clustering_use_rt_likelihood=True, within_file_mass_tol=3, within_file_rt_tol=10


To run Cluster-Match, we set *match_mode* to 1.

In [15]:
match_mode = 1
aligner_cm.run(match_mode)

Match mode 1
First stage clustering -- within_file_mass_tol=3.00, within_file_rt_tol=10.00, alpha=1.00


[Parallel(n_jobs=8)]: Done   1 out of   3 | elapsed:  4.7min remaining:  9.5min
[Parallel(n_jobs=8)]: Done   2 out of   3 | elapsed:  4.8min remaining:  2.4min
[Parallel(n_jobs=8)]: Done   3 out of   3 | elapsed:  4.8min finished


Matching precursor bins
Created 7553 clusters
Created 7579 clusters
Created 7240 clusters
Binning with mh_biggest = True
Binning with mh_biggest = True
Binning with mh_biggest = True
Assigning possible transformations 0/7553
Assigning possible transformations 0/7579
Assigning possible transformations 0/7240
Assigning possible transformations 500/7553
Assigning possible transformations 500/7579
Assigning possible transformations 500/7240
Assigning possible transformations 1000/7553
Assigning possible transformations 1000/7579
Assigning possible transformations 1000/7240
Assigning possible transformations 1500/7553
Assigning possible transformations 1500/7579
Assigning possible transformations 1500/7240
Assigning possible transformations 2000/7553
Assigning possible transformations 2000/7579
Assigning possible transformations 2000/7240
Assigning possible transformations 2500/7553
Assigning possible transformations 2500/7579
Assigning possible transformations 2500/7240
Assigning possible 

The cell below demonstrates how the alignment result from Cluster-Match can be displayed in a tabular format and saved into a *demo_cluster_match_all.csv* file. The columns in the exported table are:

- *peakset* refers to an aligned peakset (each row in the table sharing the same peakset are matched). 
- *peakset_prob* refers to the matching probability of the aligned peakset, which is always set to 1.0 in this case.
- *file* refers to the file position in the *data_list* (file 0 is the first file)
- *m/z*, *rt* and *intensity* refer to the m/z, RT and intensity of the peak
- *trans* is the MAP transformation type inferred by PrecursorCluster, while *trans_prob* is the probability of that transformation.
- *precursor mass* is the inferred precursor mass
- *members* is the number of other member peaks in the same IP cluster of this peak.

In [16]:
df = aligner_cm.print_peaksets()
display(df)
df.to_csv('demo_cluster_match_all.csv', index=False)

,peakset,peakset_prob,file,m/z,rt,intensity,trans,trans_prob,precursor mass,members
0,0,1.0,0,516.067165,420.519989,1.689182e+05,M+K,1.0000,478.11113,2
1,0,1.0,1,516.067413,421.412994,2.047615e+05,M+K,0.9990,478.11114,2
2,1,1.0,0,478.111132,421.989014,9.388497e+05,M+H,1.0000,478.11113,2
3,1,1.0,1,478.111144,421.412994,1.119185e+06,M+H,1.0000,478.11114,2
4,1,1.0,2,478.111372,424.789001,1.113332e+06,M+H,1.0000,478.11137,1
5,2,1.0,0,174.136828,842.578003,1.255721e+07,M+H,1.0000,174.13683,1
6,2,1.0,1,174.136830,842.159973,1.411731e+07,M+H,1.0000,174.13683,1
7,2,1.0,2,174.136833,842.767029,1.745757e+07,M+H,1.0000,174.13683,1
8,3,1.0,0,251.100298,641.309998,6.466016e+06,M+H,1.0000,251.10030,1
9,3,1.0,1,251.100357,641.314026,6.087699e+06,M+H,1.0000,251.10036,1


Additionally, singleton IP clusters containing only one member peak can be excluded from the output using the *exclude_singleton* parameter. In the cell below, only the non-singleton peaksets that have been aligned are displayed and exported to *demo_cluster_match_non_singleton.csv*.

In [17]:
df = aligner_cm.print_peaksets(exclude_singleton=True)
display(df)
df.to_csv('demo_cluster_match_non_singleton.csv', index=False)

,peakset,peakset_prob,file,m/z,rt,intensity,trans,trans_prob,precursor mass,members
0,0,1.0,0,516.067165,420.519989,1.689182e+05,M+K,1.0000,478.11113,2
1,0,1.0,1,516.067413,421.412994,2.047615e+05,M+K,0.9990,478.11114,2
2,1,1.0,0,478.111132,421.989014,9.388497e+05,M+H,1.0000,478.11113,2
3,1,1.0,1,478.111144,421.412994,1.119185e+06,M+H,1.0000,478.11114,2
4,10,1.0,2,393.251444,205.376007,4.942386e+05,M+NH4,1.0000,375.21756,2
5,11,1.0,2,414.180489,205.376007,3.131021e+05,M+K,1.0000,375.21756,2
6,13,1.0,1,259.027950,306.381012,8.609352e+04,M+K,1.0000,221.07251,2
7,13,1.0,2,259.027965,292.203003,9.446840e+04,M+K,0.9980,221.07257,2
8,14,1.0,1,221.072506,306.381012,6.727368e+05,M+H,1.0000,221.07251,2
9,14,1.0,2,221.072569,300.036011,4.044649e+05,M+H,1.0000,221.07257,2


## 4. Probabilistic clustering of ionization product clusters via Cluster-Cluster

The first-stage ionisation product clustering (PrecursorCluster) results from Cluster-Match are stored inside the *clustering_results* attribute. As an alternative to matching the IP clusters directly, we can further **cluster** the IP clusters. This is the Cluster-Cluster method. Essentially this procedure still performs matching since IP clusters placed into the same top-level cluster are considered to be matched (and all their member peaks too), but since this is a probabilistic method, we can now quantify the uncertainties in the matching result.

In [18]:
transformation_file = '../pos_transformations.yml'
aligner_cc = Aligner(data_list, transformation_file, hp, parallel=False)

Hyperparameters across_file_mass_tol=6, across_file_rt_tol=15, alpha_mass=1, beta=0.1, dp_alpha=1000.0, mass_clustering_n_iterations=1000, matching_alpha=0.3, rt_clustering_burnin=500, rt_clustering_nsamps=1000, second_stage_clustering_use_adduct_likelihood=True, second_stage_clustering_use_mass_likelihood=True, second_stage_clustering_use_rt_likelihood=True, within_file_mass_tol=3, within_file_rt_tol=10


To run Cluster-Cluster, we set *match_mode* to 2.

In [ ]:
match_mode = 2
aligner_cc.run(match_mode)

We can extract from Cluster-Cluster a tabular result. The output is similar to Cluster-Match, but here the column *peakset_prob* is now populated with the matching probability of the aligned peakset.

In [20]:
df = aligner_cc.print_peaksets(exclude_singleton=True)
display(df)
df.to_csv('demo_cluster_cluster_all.csv', index=False)

,peakset,peakset_prob,file,m/z,rt,intensity,trans,trans_prob,precursor mass,members
0,0,1.000,0,210.125667,264.330994,4.778752e+05,M+H,1.0000,210.12567,2
1,0,1.000,1,210.125634,262.907013,4.470650e+05,M+H,1.0000,210.12563,2
2,1,1.000,2,155.058210,355.609985,1.902087e+06,M+CH3OH+H,1.0000,123.03207,2
3,3,1.000,1,272.160564,261.294006,3.519389e+05,M+Na,1.0000,250.17790,4
4,3,1.000,0,272.160031,260.221008,4.455898e+05,M+Na,1.0000,250.17787,3
5,3,1.000,2,272.160272,258.893005,3.559621e+05,M+Na,1.0000,250.17788,4
6,4,1.000,1,460.206649,416.450012,2.135419e+05,M+2ACN+H,0.9980,378.15255,3
7,10,1.000,2,133.025206,550.732971,1.070798e+06,M+Na,1.0000,111.04329,3
8,10,1.000,0,133.025203,550.958008,1.079868e+06,M+Na,1.0000,111.04328,2
9,10,1.000,1,133.025171,551.953003,1.006336e+06,M+Na,1.0000,111.04327,2
